<a href="https://colab.research.google.com/github/arminak6/Control-Quality/blob/main/Code/anomalib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anomalib

In [2]:
!pip install lightning
!pip install kornia
!pip install FrEIA
!pip install python-dotenv
!pip install open-clip-torch
!pip install openvino
!pip install openvino-dev
!pip install onnx


  Using cached lightning-2.5.0.post0-py3-none-any.whl.metadata (40 kB)
  Using cached torchmetrics-1.6.1-py3-none-any.whl.metadata (21 kB)
  Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl.metadata (21 kB)
Using cached lightning-2.5.0.post0-py3-none-any.whl (815 kB)
Using cached torchmetrics-1.6.1-py3-none-any.whl (927 kB)
Using cached pytorch_lightning-2.5.0.post0-py3-none-any.whl (819 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for FrEIA: filename=FrEIA-0.2-py3-none-any.whl size=42762 sha256=f05f3084af111e7a1bec17fb255f5a690cb05dba9cc787bbe19d1196689b98e2
  Stored in directory: /root/.cache/pip/wheels/81/a8/e2/d532a76f72108ac4a340cbe3f86b4f591abfdbd75209a5badb
Successfully built FrEIA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
# from anomalib.data import Folder
# from anomalib.models import EfficientAd
# from anomalib.engine import Engine

# # Dataset Configuration
# datamodule = Folder(
#     name="custom_dataset",  # Add dataset name here
#     root="/content/drive/MyDrive/praeciso/anomalib_dataset",
#     normal_dir="train/good",
#     abnormal_dir="test/anomalous",
#     extensions=[".bmp"],
#     task="classification"
# )

# # Set batch sizes
# datamodule.train_batch_size = 1
# datamodule.eval_batch_size = 1

# # Initialize EfficientAd Model and Engine
# model = EfficientAd()
# engine = Engine(max_epochs=5)

# # Train the model
# engine.fit(datamodule=datamodule, model=model)


In [28]:
from anomalib.data import Folder
from anomalib.models import EfficientAd
from anomalib.engine import Engine
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import CSVLogger
import pandas as pd
import matplotlib.pyplot as plt

# Dataset Configuration
datamodule = Folder(
    name="custom_dataset",
    root="/content/drive/MyDrive/praeciso/anomalib_dataset",
    normal_dir="train/good",
    abnormal_dir="test/anomalous",
    extensions=[".bmp"],
    task="classification"
)

# Set batch sizes
datamodule.train_batch_size = 1
datamodule.eval_batch_size = 1

# Initialize Model and Callbacks
model = EfficientAd()

# Define Model Checkpoint Callback
checkpoint_callback = ModelCheckpoint(
    monitor="image_AUROC",  # Save the model with the highest AUROC
    mode="max",
    save_top_k=1
)

# Define Early Stopping Callback
early_stopping_callback = EarlyStopping(
    monitor="image_AUROC",  # Monitor AUROC instead of val_loss
    patience=5,
    mode="max",
    verbose=True
)

# Define CSV Logger for Logging Metrics
csv_logger = CSVLogger("logs", name="EfficientAd_training")

# Initialize Engine with Callbacks and Logger
engine = Engine(
    max_epochs=50,
    accelerator="gpu",
    devices=1,
    callbacks=[checkpoint_callback, early_stopping_callback],
    logger=csv_logger
)

# Train the Model
engine.fit(datamodule=datamodule, model=model)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: 
  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model       

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/usr/local/lib/python3.10/dist-packages/torchv

Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:05<00:00,  3.64it/s]
INFO: Metric image_AUROC improved. New best score: 0.417
INFO:lightning.pytorch.callbacks.early_stopping:Metric image_AUROC improved. New best score: 0.417


...

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:04<00:00,  3.91it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:04<00:00,  3.93it/s]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 19/19 [00:02<00:00,  6.90it/s]
INFO: Monitored metric image_AUROC did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric image_AUROC did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.


In [33]:
# Evaluate the best checkpoint after training
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path
)

print("Test Results:", test_results)

import os
import pandas as pd
import matplotlib.pyplot as plt

# Load training logs
log_file = "logs/EfficientAd_training/version_0/metrics.csv"
logs = pd.read_csv(log_file)

# Directory to save plots
output_dir = "/content/drive/MyDrive/praeciso/show"
os.makedirs(output_dir, exist_ok=True)

# Check available metrics
print("Available metrics in logs:")
print(logs.columns)

# Plot AUROC if available
if "image_AUROC" in logs:
    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["image_AUROC"], label="Image AUROC", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("AUROC")
    plt.title("Image AUROC During Training")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "image_auroc.png"))  # Save plot
    plt.close()  # Close the figure

# Plot F1 Score if available
if "image_F1Score" in logs:
    plt.figure(figsize=(10, 5))
    plt.plot(logs["epoch"], logs["image_F1Score"], label="Image F1 Score", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("F1 Score")
    plt.title("Image F1 Score During Training")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "image_f1_score.png"))  # Save plot
    plt.close()  # Close the figure

# Plot train_loss or val_loss if available
if "train_loss" in logs or "val_loss" in logs:
    plt.figure(figsize=(10, 5))
    if "train_loss" in logs:
        plt.plot(logs["epoch"], logs["train_loss"], label="Training Loss", marker="o")
    if "val_loss" in logs:
        plt.plot(logs["epoch"], logs["val_loss"], label="Validation Loss", marker="o")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(output_dir, "loss_curve.png"))  # Save plot
    plt.close()  # Close the figure

print(f"Plots saved successfully in {output_dir}")


/usr/local/lib/python3.10/dist-packages/anomalib/engine/engine.py:391: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=model.de

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │     0.95652174949646      │
└───────────────────────────┴───────────────────────────┘

Test Results: [{'image_AUROC': 1.0, 'image_F1Score': 0.95652174949646}]
Available metrics in logs:
Index(['epoch', 'image_AUROC', 'image_F1Score', 'step', 'train_ae_step',
       'train_loss_step', 'train_st_step', 'train_stae_step'],
      dtype='object')
Plots saved successfully in /content/drive/MyDrive/praeciso/show


In [36]:
import os
import matplotlib.pyplot as plt

# Directory to save plots
output_dir = "/content/drive/MyDrive/praeciso/show"
os.makedirs(output_dir, exist_ok=True)

# Check available metrics and handle missing data
if "epoch" in logs.columns:
    # Filter rows with non-NaN values for AUROC
    if "image_AUROC" in logs:
        valid_logs_auroc = logs[~logs["image_AUROC"].isna()]
        if not valid_logs_auroc.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_auroc["epoch"], valid_logs_auroc["image_AUROC"], label="Image AUROC", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("AUROC")
            plt.title("Image AUROC During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "image_auroc_fixed.png"))
            plt.close()
        else:
            print("No valid data for Image AUROC found in logs.")

    # Filter rows with non-NaN values for F1 Score
    if "image_F1Score" in logs:
        valid_logs_f1 = logs[~logs["image_F1Score"].isna()]
        if not valid_logs_f1.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_f1["epoch"], valid_logs_f1["image_F1Score"], label="Image F1 Score", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("F1 Score")
            plt.title("Image F1 Score During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "image_f1_score_fixed.png"))
            plt.close()
        else:
            print("No valid data for Image F1 Score found in logs.")

    # Plot training loss if available
    if "train_loss_step" in logs:
        valid_logs_loss = logs[~logs["train_loss_step"].isna()]
        if not valid_logs_loss.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(valid_logs_loss["epoch"], valid_logs_loss["train_loss_step"], label="Training Loss", marker="o")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.title("Training Loss During Training")
            plt.legend()
            plt.grid()
            plt.savefig(os.path.join(output_dir, "train_loss_fixed.png"))
            plt.close()
        else:
            print("No valid data for Training Loss found in logs.")
else:
    print("`epoch` column not found in logs.")

print(f"Plots saved successfully in {output_dir}")


Plots saved successfully in /content/drive/MyDrive/praeciso/show


In [34]:
# Evaluate the best checkpoint after training
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path
)

print(test_results)


/usr/local/lib/python3.10/dist-packages/anomalib/engine/engine.py:391: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=model.de

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │     0.95652174949646      │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 1.0, 'image_F1Score': 0.95652174949646}]


In [ ]:
# Print test results
print("Test Results:")
for key, value in test_results.items():
    print(f"{key}: {value}")


In [13]:
import random
import os
import matplotlib.pyplot as plt

# Set the number of samples to visualize
num_samples = 10

# Create directory for saving visualizations
output_dir = "random_visualizations"
os.makedirs(output_dir, exist_ok=True)

# Load the test dataset
test_loader = datamodule.test_dataloader()

# Collect all images labeled as defected (anomalous)
defected_images = []
defected_predictions = []

for batch, prediction in zip(test_loader, engine.trainer.predict(model=model, dataloaders=test_loader)):
    # Check if the batch contains anomaly samples
    labels = batch["label"].detach().cpu().numpy()  # Get labels (0: normal, 1: anomalous)
    for i in range(len(labels)):
        if labels[i] == 1:  # Select only anomalous images
            defected_images.append(batch["image"][i])
            defected_predictions.append(prediction["anomaly_maps"][i])

# Randomly select 10 defected images
random_indices = random.sample(range(len(defected_images)), min(num_samples, len(defected_images)))
selected_images = [defected_images[i] for i in random_indices]
selected_predictions = [defected_predictions[i] for i in random_indices]

# Visualize the selected images and their anomaly maps
for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    # Normalize and adjust anomaly map shape
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    image = image.permute(1, 2, 0).cpu().numpy()  # Adjust image for visualization

    # Plot original image and anomaly map
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title(f"Original Image {idx+1}")
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title(f"Anomaly Map {idx+1}")
    plt.imshow(anomaly_map, cmap="jet")
    plt.axis("off")

    plt.show()

    # Save visualizations
    plt.imsave(f"{output_dir}/image_{idx+1}.png", image)
    plt.imsave(f"{output_dir}/anomaly_map_{idx+1}.png", anomaly_map, cmap="jet")


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Directory to save heatmap visualizations
heatmap_dir = "heatmap_visualizations"
os.makedirs(heatmap_dir, exist_ok=True)

for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    # Normalize and reshape the anomaly map
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    # Convert image to numpy for overlay
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)  # Convert to 8-bit image

    # Resize anomaly map to match the image size
    anomaly_map_resized = cv2.resize(anomaly_map, (image_np.shape[1], image_np.shape[0]))

    # Generate the heatmap
    heatmap = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Overlay the heatmap onto the original image
    overlay = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0)

    # Plot original image, anomaly map, and heatmap overlay
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title(f"Original Image {idx+1}")
    plt.imshow(image_np)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title(f"Anomaly Map {idx+1}")
    plt.imshow(anomaly_map, cmap="jet")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title(f"Heatmap Overlay {idx+1}")
    plt.imshow(overlay)
    plt.axis("off")

    plt.show()

    # Save heatmap visualization
    plt.imsave(f"{heatmap_dir}/heatmap_{idx+1}.png", overlay)


<ipython-input-14-afe692da1d03>:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(15, 5))


In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Directory to save segmentation masks
segmentation_dir = "segmentation_visualizations"
os.makedirs(segmentation_dir, exist_ok=True)

# Set a threshold value for segmentation
threshold_value = 0.5  # Adjust based on model outputs and dataset characteristics

for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    # Normalize and reshape the anomaly map
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    # Threshold the anomaly map to create a binary segmentation mask
    binary_mask = (anomaly_map > threshold_value).astype(np.uint8)

    # Resize binary mask to match the image size
    binary_mask_resized = cv2.resize(binary_mask, (image.shape[2], image.shape[1]), interpolation=cv2.INTER_NEAREST)

    # Convert image to numpy for overlay
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)  # Convert to 8-bit image

    # Overlay the segmentation mask on the original image
    overlay = cv2.addWeighted(image_np, 0.7, cv2.applyColorMap(binary_mask_resized * 255, cv2.COLORMAP_JET), 0.3, 0)

    # Plot the original image, anomaly map, and segmentation mask
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title(f"Original Image {idx+1}")
    plt.imshow(image_np)
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.title(f"Anomaly Map {idx+1}")
    plt.imshow(anomaly_map, cmap="jet")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.title(f"Segmentation Mask {idx+1}")
    plt.imshow(overlay)
    plt.axis("off")

    plt.show()

    # Save segmentation visualization
    plt.imsave(f"{segmentation_dir}/segmentation_{idx+1}.png", overlay)


# **ALL together**

In [17]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

# Directory to save outputs
output_dir = "combined_visualizations"
os.makedirs(output_dir, exist_ok=True)

# Set threshold for segmentation
threshold_value = 0.5  # Adjust this based on your data

# Set the number of images to process
num_samples = 10

# Randomly select images from defected dataset
test_loader = datamodule.test_dataloader()
defected_images = []
defected_predictions = []

for batch, prediction in zip(test_loader, engine.trainer.predict(model=model, dataloaders=test_loader)):
    labels = batch["label"].detach().cpu().numpy()  # Get labels (0: normal, 1: anomalous)
    for i in range(len(labels)):
        if labels[i] == 1:  # Anomalous images only
            defected_images.append(batch["image"][i])
            defected_predictions.append(prediction["anomaly_maps"][i])

# Randomly select 10 images if available
random_indices = random.sample(range(len(defected_images)), min(num_samples, len(defected_images)))
selected_images = [defected_images[i] for i in random_indices]
selected_predictions = [defected_predictions[i] for i in random_indices]

# Process each selected image
for idx, (image, anomaly_map) in enumerate(zip(selected_images, selected_predictions)):
    unique_id = f"{idx+1:03d}"  # Unique number for each image

    # Normalize anomaly map
    anomaly_map = anomaly_map.squeeze().detach().cpu().numpy()
    anomaly_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min())

    # Convert image for visualization
    image_np = image.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)

    # Create heatmap
    anomaly_map_resized = cv2.resize(anomaly_map, (image_np.shape[1], image_np.shape[0]))
    heatmap = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)
    heatmap_overlay = cv2.addWeighted(image_np, 0.7, heatmap, 0.3, 0)

    # Create segmentation mask
    binary_mask = (anomaly_map > threshold_value).astype(np.uint8)
    binary_mask_resized = cv2.resize(binary_mask, (image_np.shape[1], image_np.shape[0]), interpolation=cv2.INTER_NEAREST)
    segmentation_overlay = cv2.addWeighted(image_np, 0.7, cv2.applyColorMap(binary_mask_resized * 255, cv2.COLORMAP_JET), 0.3, 0)

    # Apply colormap to anomaly map before saving
    anomaly_map_colored = cv2.applyColorMap((anomaly_map_resized * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Save all outputs
    cv2.imwrite(os.path.join(output_dir, f"image_{unique_id}.png"), cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR))
    cv2.imwrite(os.path.join(output_dir, f"anomaly_map_{unique_id}.png"), anomaly_map_colored)  # Save as colored map
    cv2.imwrite(os.path.join(output_dir, f"heatmap_{unique_id}.png"), heatmap_overlay)
    cv2.imwrite(os.path.join(output_dir, f"segmentation_{unique_id}.png"), segmentation_overlay)


    # Optional: Display outputs
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.title("Original Image")
    plt.imshow(image_np)
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.title("Anomaly Map")
    plt.imshow(anomaly_map, cmap="jet")
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.title("Heatmap Overlay")
    plt.imshow(heatmap_overlay)
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.title("Segmentation Map")
    plt.imshow(segmentation_overlay)
    plt.axis("off")

    plt.show()


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [19]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Run predictions on the test set
test_loader = datamodule.test_dataloader()
predictions = engine.trainer.predict(model=model, dataloaders=test_loader)

y_true = []
y_pred = []

# Extract true labels and predictions
for batch, prediction in zip(test_loader, predictions):
    labels = batch["label"].cpu().numpy()  # Ground-truth labels (0: normal, 1: anomalous)
    anomaly_scores = prediction["pred_scores"].detach().cpu().numpy()  # Model's anomaly scores

    # Convert anomaly scores to binary predictions (using a threshold, e.g., 0.5)
    binary_predictions = (anomaly_scores > 0.5).astype(int)

    y_true.extend(labels)
    y_pred.extend(binary_predictions)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

print("Confusion Matrix:")
print(cm)

# Classification report (includes precision, recall, F1-score)
report = classification_report(y_true, y_pred, target_names=["Normal", "Anomalous"])
print("Classification Report:")
print(report)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Confusion Matrix:
[[ 8  0]
 [ 2 10]]
Classification Report:
              precision    recall  f1-score   support

      Normal       0.80      1.00      0.89         8
   Anomalous       1.00      0.83      0.91        12

    accuracy                           0.90        20
   macro avg       0.90      0.92      0.90        20
weighted avg       0.92      0.90      0.90        20



In [22]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Run predictions on the test set
test_loader = datamodule.test_dataloader()
predictions = engine.trainer.predict(model=model, dataloaders=test_loader)

y_true = []
y_pred = []

# Extract true labels and predictions
for batch, prediction in zip(test_loader, predictions):
    labels = batch["label"].cpu().numpy()  # Ground-truth labels (0: normal, 1: anomalous)
    anomaly_scores = prediction["pred_scores"].detach().cpu().numpy()  # Model's anomaly scores

    # Convert anomaly scores to binary predictions (using a threshold, e.g., 0.5)
    binary_predictions = (anomaly_scores > 0.5).astype(int)

    y_true.extend(labels)
    y_pred.extend(binary_predictions)

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Create a confusion matrix plot
classes = ["Normal", "Anomalous"]  # Replace with your class labels
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)

# Add a title
plt.title("Confusion Matrix")
plt.show()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [23]:
import seaborn as sns
import pandas as pd

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=["Normal", "Anomalous"], columns=["Normal", "Anomalous"])

# Plot confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues", cbar=True)
plt.title("Confusion Matrix (Reordered)")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()


In [20]:
from sklearn.metrics import precision_recall_fscore_support

thresholds = np.linspace(0, 1, 100)
best_f1 = 0
best_threshold = 0

for threshold in thresholds:
    binary_predictions = (np.array(y_pred) > threshold).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, binary_predictions, average="binary")
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"Best Threshold: {best_threshold}")
print(f"Best F1-Score: {best_f1}")


Best Threshold: 0.0
Best F1-Score: 0.9090909090909091


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt

# Calculate ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic")
plt.legend(loc="lower right")
plt.show()

# Calculate Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_true, y_pred)
pr_auc = auc(recall, precision)

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, color="darkorange", lw=2, label=f"PR curve (area = {pr_auc:.2f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend(loc="lower left")
plt.show()


In [ ]:
from anomalib.deploy import ExportType

engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/torch_model.py:28: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mean = torch.tensor([0.485, 0.456, 0.406])[None, :, None, None].to(x.device)
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/torch_model.py:29: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  std = torch.tensor([0.229, 0.224, 0.225])[None, :, None, None].to(x.device)
/usr/local/lib/python3.10/dist-packages/anomalib/

PosixPath('/content/results/EfficientAd/custom_dataset/latest/weights/openvino/model.xml')

In [ ]:
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import os

# 1. Read .bmp image using PIL
image_path = "/content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_00.bmp"
image = Image.open(image_path)

# 2. Convert and save as .png
output_path = image_path.replace(".bmp", ".png")
image.save(output_path)

# 3. Read the newly saved .png image as tensor
image_tensor = T.ToTensor()(Image.open(output_path))

# 4. Convert to PIL for visualization
transform = T.ToPILImage()
image_pil = transform(image_tensor)

# 5. Handle grayscale visualization
if image_tensor.shape[0] == 1:  # Grayscale check
    plt.imshow(image_pil, cmap='gray')  # Force grayscale colormap
else:
    plt.imshow(image_pil)

plt.axis('off')
plt.show(block=True)

# 6. Print image shape for debugging (optional)
print(f"Image shape: {image_tensor.shape}")
print(f"Converted image saved at: {output_path}")


Image shape: torch.Size([1, 1536, 2048])
Converted image saved at: /content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_00.png


In [ ]:
from pathlib import Path

output_path = Path(engine.trainer.default_root_dir)  # Get model directory
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"

# Print paths and check existence
print("Model path:", openvino_model_path)
print("Metadata path:", metadata)
print("Exists?", openvino_model_path.exists(), metadata.exists())


Model path: /content/results/EfficientAd/custom_dataset/latest/weights/openvino/model.bin
Metadata path: /content/results/EfficientAd/custom_dataset/latest/weights/openvino/metadata.json
Exists? True True


In [ ]:
from anomalib.deploy import OpenVINOInferencer

# Set the path to your exported OpenVINO model
openvino_model_path = "/content/drive/MyDrive/praeciso/exported_model.xml"

# Initialize the OpenVINO Inferencer
metadata = model.metadata if hasattr(model, 'metadata') else None

inferencer = OpenVINOInferencer(
    path=openvino_model_path,
    metadata=metadata,
    device="CPU"
)



RuntimeError: Exception from src/inference/src/cpp/core.cpp:90:
Check 'util::directory_exists(path) || util::file_exists(path)' failed at src/frontends/common/src/frontend.cpp:113:
FrontEnd API failed with GeneralFailure:
ir: Could not open the file: "/content/drive/MyDrive/praeciso/exported_model.xml"



In [ ]:
import os

path = "/content/drive/MyDrive/praeciso/exported_model.xml"
print(f"File exists: {os.path.isfile(path)}")


File exists: False


In [ ]:
# Convert grayscale image to RGB (3 channels)
image_for_inference = Path("/content/drive/MyDrive/praeciso/anomalib_dataset/test/anomalous/KO1_30K_97_00.png")
image = Image.open(image_for_inference)

# If image is grayscale, convert to RGB
if image.mode != 'RGB':
    image = image.convert('RGB')
    image.save(image_for_inference)  # Overwrite or save as a new image

# Run prediction with the fixed image
predictions = inferencer.predict(image=image_for_inference)

# Display anomaly score and label
print(f"Prediction Score: {predictions.pred_score}")
print(f"Prediction Label: {predictions.pred_label}")


Prediction Score: 0.5018951558223842
Prediction Label: 1


In [ ]:
# Visualize original image
plt.figure()
plt.imshow(predictions.image)
plt.title("Original Image")
plt.axis('off')

# Visualize anomaly map
plt.figure()
plt.imshow(predictions.anomaly_map, cmap='jet')
plt.title("Anomaly Map")
plt.axis('off')

# Visualize heatmap (overlay)
plt.figure()
plt.imshow(predictions.heat_map)
plt.title("Heat Map (Overlay)")
plt.axis('off')

plt.show()


In [ ]:
from PIL import Image
import numpy as np

# Save anomaly map manually
anomaly_map = predictions.anomaly_map  # Get the anomaly map (numpy array)

# Normalize and convert to 8-bit for saving
normalized_map = (anomaly_map - anomaly_map.min()) / (anomaly_map.max() - anomaly_map.min() + 1e-6)
anomaly_map_8bit = (normalized_map * 255).astype(np.uint8)

# Convert to image and save
anomaly_map_image = Image.fromarray(anomaly_map_8bit)
anomaly_map_image.save(image_for_inference.with_name("anomaly_map.png"))

# Save heatmap (optional)
heatmap = predictions.heat_map
heatmap_image = Image.fromarray((heatmap * 255).astype(np.uint8))
heatmap_image.save(image_for_inference.with_name("heatmap.png"))

print("Anomaly map and heatmap saved.")


Anomaly map and heatmap saved.


In [ ]:
# Load and show the saved anomaly map
saved_anomaly_map = Image.open(image_for_inference.with_name("anomaly_map.png"))
plt.figure()
plt.imshow(saved_anomaly_map, cmap='gray')
plt.title("Saved Anomaly Map")
plt.axis('off')

# Load and show the saved heatmap
saved_heatmap = Image.open(image_for_inference.with_name("heatmap.png"))
plt.figure()
plt.imshow(saved_heatmap)
plt.title("Saved Heat Map")
plt.axis('off')

plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
from pathlib import Path
from PIL import Image

# Initialize lists to store predictions and ground truth
y_true = []
y_pred = []

# Path to test images
test_dir = Path("/content/drive/MyDrive/praeciso/anomalib_dataset/test")

# Iterate over normal and anomalous folders
for label, subfolder in enumerate(["good", "anomalous"]):
    test_images = list((test_dir / subfolder).glob("*.png"))

    for img_path in test_images:
        # Read image using PIL
        image = Image.open(img_path)

        # Convert to RGB if the image is grayscale
        if image.mode != 'RGB':
            image = image.convert('RGB')
            image.save(img_path)  # Overwrite or save it

        # Run inference on the converted image
        predictions = inferencer.predict(image=img_path)

        # Store ground truth (0 for normal, 1 for anomalous)
        y_true.append(label)

        # Store predicted label
        y_pred.append(predictions.pred_label)

# Calculate Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Model Accuracy on Test Set: {accuracy * 100:.2f}%")


Model Accuracy on Test Set: 66.67%
